### torchvision Pretrained models
* torchvision은 많은 CNN 최신 모델들을 가지고 있으며, 이들 모델들을 ImageNet 데이터 세트를 학습하여 Weight를 가지고 있음. 
* 이들 최신 모델을 이용하여 간단하게 Custom 모델을 만들 수 있음.
* https://pytorch.org/vision/main/models.html 에서 모델 리스트를 확인해 볼 수 있음.

### torchvision의 models 모듈 활용하기

In [ ]:
import torch
import torchvision
from torchvision import models

print(torchvision.__version__)

# 현 torchvision 버전에서 사용 가능한 model들의 리스트
models.list_models() # include 또는 exclude로 model 리스트 필터링 

In [ ]:
# 특정 모델이 가지고 있는 weight의 종류를 확인. 하나의 모델은 학습 데이터 세트, 학습 방식에 따라 여러개의 weights를 가질 수 있음. 
weights_enum = models.get_model_weights(name='resnet50') #'efficientnet_b1'
print([weight for weight in weights_enum])

In [ ]:
# 모든 모델이 가지고 있는 weights 들의 list 출력
all_models = models.list_models()
for model_name in all_models:
    weights = models.get_model_weights(name=model_name)
    if weights:
        print(f"Model: {model_name}, Weights: {[weight for weight in weights]}")

### torchvision models의 Pretrained 모델 로드하기
* 과거 버전에서는 models.model명(pretrained=True)를 이용하여 pretrained 학습된 모델을 가져올 수 있음.
* 현 버전에서는 동일 모델이더라도 pretrained된 weight를 지정해서 모델을 생성하도록 변경됨. 

In [ ]:
#과거 버전은 pretrained=True를 이용하여 미리 pretrained된 weight가 설정된 모델을 가져올 수 있었지만, torchvision 0.13 부터 deprecated됨. 
model_temp = models.vgg19_bn(pretrained=True) #pretrained=False이면 pretrained된 weight를 가져오지 않음. resnet50, efficientnet-b0, resnext101_32x8d
print(model_temp)

In [ ]:
print([weight for weight in models.get_model_weights('vgg19_bn')])

In [ ]:
# from torchvision.models import VGG19_Weights, VGG19_BN_Weights

# batch normalization이 적용된 모델 가져오기
print([weight for weight in models.get_model_weights('vgg19_bn')])
vgg_model = models.vgg19_bn(weights=models.VGG19_BN_Weights.IMAGENET1K_V1) #weights='IMAGENET1K_V1' 또는 'DEFAULT'
print(vgg_model)

In [ ]:
from torchinfo import summary

summary(model=vgg_model, input_size=(1, 3, 418, 418), #(1, 3, 32, 32)
        col_names=['input_size', 'output_size', 'num_params'], 
        row_settings=['var_names'])

In [ ]:
# resnet 모델 가져오기
print([weight for weight in models.get_model_weights('resnet50')])
resnet_model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)#weights='IMAGENET1K_V2' 또는 'DEFAULT'
print(resnet_model)

### Classfication Layer를 변경하여 Custom 모델 만들기
* torchvision의 CNN 분류 모델들은 대부분 ImageNet 데이터를 판별하기 위해 만들어짐. ImageNet 데이터는 Class의 갯수가 1000개 이므로 최종 classification layer의 output 이 1000개로 되어 있음.
* Class의 갯수가 1000개가 아닌 Custom Dataset을 판별하는 모델을 위해서는 마지막 Classification Layer의 output을 Class의 갯수에 맞춰서 customization 하여 모델을 생성할 필요가 있음.
* torchvision models의 최종 classifier layer(또는 block)의 변수명은 (거의 대부분)classifier 또는 fc이며, 이들 변수명에 직접 custom classifier layer를 할당하여 변경 

In [ ]:
vgg_model.classifier # resnet_model.fc
# resnet_model.fc

### Sequential block으로 classifier가 설정되어 있는 경우
* 맨 마지막 Classifier Linear Layer만 출력 features를 class 갯수로 가지는 Layer로 변경
* 전체 Sequential Block을 새롭게 교체

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

NUM_CLASSES = 10

# 직접 model의 최종 classifier 지정하여 변경할 수 있음.
vgg_model.classifier = nn.Sequential(
      nn.Linear(in_features=25088, out_features=4096, bias=True),
      nn.ReLU(inplace=True),
      nn.Dropout(p=0.5, inplace=False),
      nn.Linear(in_features=4096, out_features=4096, bias=True),
      nn.ReLU(inplace=True),
      nn.Dropout(p=0.5, inplace=False), 
      nn.Linear(in_features=4096, out_features=NUM_CLASSES, bias=True)
)

# Sequential을 새롭게 정의
# vgg_model.classifier = nn.Sequential(
#     nn.Linear(in_features=25088, out_features=4096, bias=True),
#     nn.ReLU(inplace=True),
#     nn.Dropout(p=0.5, inplace=False),
#     nn.Linear(in_features=4096, out_features=500, bias=True),
#     nn.ReLU(inplace=True),
#     nn.Dropout(p=0.5, inplace=False),
#     nn.Linear(in_features=500, out_features=NUM_CLASSES, bias=True)
# )

print(vgg_model)

In [ ]:
vgg_model.classifier[6]

In [ ]:
NUM_CLASSES = 10
# Sequential 내에서 최종 layer만 변경. 
#vgg_model.classifier[6] = nn.Linear(in_features=4096, out_features=NUM_CLASSES)
# 보통은 아래와 같이 in_features값을 최종 layer의 in_features로 직접 가져와서 생성.
num_in_features = vgg_model.classifier[6].in_features
vgg_model.classifier[6] == nn.Linear(in_features=num_in_features, out_features=NUM_CLASSES)
print(vgg_model)

In [ ]:
# resnet의 경우 최종 classifier의 변수명이 fc임. out_features가 10인 Linear Layer로 변경. 
num_in_features = resnet_model.fc.in_features
resnet_model.fc = nn.Linear(in_features=num_in_features, out_features=NUM_CLASSES)
#resnet_model.fc = nn.Linear(in_features=2048, out_features=10, bias=True)
print(resnet_model)

In [ ]:
# 또는 아예 최종 layer 구조를 변경
resnet_model.fc = nn.Sequential(
                    nn.Dropout(0.5),
                    nn.Linear(in_features=2048, out_features=512),
                    nn.ReLU(),
                    nn.Dropout(0.3),
                    nn.Linear(in_features=512, out_features=128),
                    nn.ReLU(),
                    nn.Dropout(0.2),
                    nn.Linear(in_features=128, out_features=NUM_CLASSES)
)
print(resnet_model)

### Pretrained Model(Weights)에 적용된 기본 transforms 확인하기
* Weights 객체에 transforms() 메소드를 호출하여 pretrained 모델에 적용된 기본 transforms들을 확인
* Pretrained Model에서 inference 수행시 입력 이미지 사이즈 설정시 활용

In [ ]:
from torchvision import models

resnet_weights = models.ResNet50_Weights.IMAGENET1K_V2# 또는 ResNet50_Weights.DEFAULT
resnet_transforms = resnet_weights.transforms()

print(resnet_transforms)

### 개와 고양이 판별 모델을 Pretrained Model을 기반으로 수행. 

#### Custom Model 생성 함수 만들기

In [ ]:
import torch
import torch.nn as nn
from torchvision import models

def create_vgg_model(model_name, num_classes=10):
    model = None
    if model_name == 'vgg16_bn':
        model = models.vgg16_bn(weights=models.VGG16_BN_Weights.IMAGENET1K_V1)
    elif model_name == 'vgg19_bn':
        model = models.vgg19_bn(weights=models.VGG19_BN_Weights.IMAGENET1K_V1)
    
    num_in_features = model.classifier[6].in_features
    model.classifier[6] = nn.Linear(in_features=num_in_features, out_features=num_classes)

    return model

def create_resnet_model(model_name, num_classes=10):
    model = None
    if model_name == 'resnet50':
        model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)
    elif model_name == 'resnet101':
        model = models.resnet101(weights=models.ResNet101_Weights.IMAGENET1K_V2)
    
    num_in_features = model.fc.in_features
    model.fc = nn.Linear(in_features=num_in_features, out_features=num_classes)

    return model
    
resnet_model = create_resnet_model('resnet50', num_classes=2) #resnet50, resnet101
print(resnet_model)

# summary(model=resnet_model, input_size=(1, 3, 224, 224), #(1, 3, 64, 64)
#         col_names=['input_size', 'output_size', 'num_params'], 
#         row_settings=['var_names'])

### 개와 고양이 Dataset 및 DataLoader 생성

#### 개와 고양이 데이터 다운로드 및 메타 데이터 생성

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from sklearn.model_selection import train_test_split

paths = [] # 이미지 파일 경로 리스트
dataset_gubuns = [] # train/test 구분 리스트
label_gubuns = [] #강아지/고양이 리스트

# os.walk()를 이용하여 특정 디렉토리 밑에 있는 모든 하위 디렉토리를 모두 조사. 
# cat-and-dog 하위 디렉토리 밑에 jpg 확장자를 가진 파일이 모두 이미지 파일임
# cat-and-dog 밑으로 /train/, /test/ 하위 디렉토리 존재(학습, 테스트 용 이미지 파일들을 가짐)
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        # 이미지 파일이 아닌 파일도 해당 디렉토리에 있음.
        if '.jpg' in filename:
            # 파일의 절대 경로를 file_path 변수에 할당. 
            file_path = dirname+'/'+ filename
            paths.append(file_path)
            # 파일의 절대 경로에 training_set, test_set가 포함되어 있으면 데이터 세트 구분을 'train'과 'test'로 분류. 
            if '/training_set/' in file_path:
                dataset_gubuns.append('train')  
            elif '/test_set/' in file_path:
                dataset_gubuns.append('test')
            else: dataset_gubuns.append('N/A')
            
            # 파일의 절대 경로에 dogs가 있을 경우 해당 파일은 dog 이미지 파일이고, cats일 경우는 cat 이미지 파일임. 
            if 'dogs' in file_path:
                label_gubuns.append('DOG')
            elif 'cats' in file_path:
                label_gubuns.append('CAT')
            else: label_gubuns.append('N/A')
# DataFrame 메타 데이터 생성. 
data_df = pd.DataFrame({'path':paths, 
                        'dataset':dataset_gubuns, 
                        'label':label_gubuns})
# Target값 0, 1 변환
label_mapping = {'DOG': 0, 'CAT': 1}
data_df['target'] = data_df['label'].map(label_mapping)

# 전체 데이터 세트에서 학습과 테스트용 메타 정보 DataFrame 생성. 
train_df = data_df[data_df['dataset']=='train']
test_df = data_df[data_df['dataset']=='test']
# 기존 학습 DataFrame을 다시 학습과 검증 DataFrame으로 분할. 85%가 학습, 15%가 검증 
tr_df, val_df = train_test_split(train_df, test_size=0.15, stratify=train_df['target'], random_state=2025)

print(data_df.shape, train_df.shape, tr_df.shape, val_df.shape)

#### Dataset 및 학습/검증용 DataLoader 생성

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms as T
from PIL import Image

class CnD_Dataset(Dataset):
    # 이미지 파일리스트, 타겟 파일리스트, transforms 등 이미지와 타겟 데이터 가공에 필요한 인자들을 입력 받음
    def __init__(self, image_paths, targets=None, transform=None):
        self.image_paths = image_paths
        self.targets = targets
        self.transform = transform
    
    # 전체 건수를 반환
    def __len__(self):
        return len(self.image_paths)
        
    # idx로 지정된 하나의 image, label을 tensor 형태로 반환
    def __getitem__(self, idx):    
        # PIL을 이용하여 이미지 로딩하고 PIL Image 객체 반환.
        pil_image = Image.open(self.image_paths[idx])
        # 보통은 transform이 None이 되는 경우는 거의 없음(Tensor 변환이라도 있음)
        image = self.transform(pil_image)

        if self.targets is not None:
            # 개별 target값을 tensor로 변환.
            target = torch.tensor(self.targets[idx])
            return image, target
        # 테스트 데이터의 경우 targets가 입력 되지 않을 수 있으므로 이를 대비. 
        else:
            return image

In [ ]:
from torch.utils.data import DataLoader
from torchvision import transforms as T

BATCH_SIZE = 16
IMG_SIZE = 224
IMG_MEANS = [0.485, 0.456, 0.406] # ImageNet 데이터세트의 이미지 채널별 평균값
IMG_STD = [0.229, 0.224, 0.225] # ImageNet 데이터세트의 이미지 채널별 표준편차값

def create_tr_val_loader(tr_df, val_df, transform):
    tr_dataset = CnD_Dataset(image_paths=tr_df['path'].to_list(), 
                            targets=tr_df['target'].to_list(), transform=transform)
    val_dataset = CnD_Dataset(image_paths=val_df['path'].to_list(), 
                            targets=val_df['target'].to_list(), transform=transform)
    tr_loader = DataLoader(tr_dataset, batch_size = BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=2*BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)

    return tr_loader, val_loader

transform_01 = T.Compose([
            T.Resize(size=(IMG_SIZE, IMG_SIZE)),
            T.ToTensor(), T.Normalize(mean=IMG_MEANS, std=IMG_STD)
])

tr_loader, val_loader = create_tr_val_loader(tr_df=tr_df, val_df=val_df, transform=transform_01)

### Trainer Class 생성 

In [ ]:
from tqdm import tqdm
import torch.nn.functional as F

class Trainer:
    def __init__(self, model, loss_fn, optimizer, train_loader, val_loader, scheduler=None, device=None):
        self.model = model.to(device)
        self.loss_fn = loss_fn
        self.optimizer = optimizer
        self.train_loader = train_loader
        self.val_loader = val_loader
        # scheduler 추가
        self.scheduler = scheduler
        self.device = device
        # 현재 learning rate 변수 추가
        self.current_lr = self.optimizer.param_groups[0]['lr']

    def train_epoch(self, epoch):
        self.model.train()

        # running 평균 loss 계산.
        accu_loss = 0.0
        running_avg_loss = 0.0
        # 정확도, 정확도 계산을 위한 전체 건수 및 누적 정확건수
        num_total = 0.0
        accu_num_correct = 0.0
        accuracy = 0.0
        # tqdm으로 실시간 training loop 진행 상황 시각화
        with tqdm(total=len(self.train_loader), desc=f"Epoch {epoch+1} [Training..]", leave=True) as progress_bar:
            for batch_idx, (inputs, targets) in enumerate(self.train_loader):
                # 반드시 to(self.device). to(device) 아님.
                inputs = inputs.to(self.device)
                targets = targets.to(self.device)

                # Forward pass
                outputs = self.model(inputs)
                loss = self.loss_fn(outputs, targets)

                # Backward pass
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()

                # batch 반복 시 마다 누적  loss를 구하고 이를 batch 횟수로 나눠서 running 평균 loss 구함.
                accu_loss += loss.item()
                running_avg_loss = accu_loss /(batch_idx + 1)

                # accuracy metric 계산
                # outputs 출력 예측 class값과 targets값 일치 건수 구하고
                num_correct = (outputs.argmax(-1) == targets).sum().item()
                # 배치별 누적 전체 건수와 누적 전체 num_correct 건수로 accuracy 계산
                num_total += inputs.shape[0]
                accu_num_correct += num_correct
                accuracy = accu_num_correct / num_total

                #tqdm progress_bar에 진행 상황 및 running 평균 loss와 정확도 표시
                progress_bar.update(1)
                if batch_idx % 20 == 0 or (batch_idx + 1) == progress_bar.total:  # 20 batch 횟수마다 또는 맨 마지막 batch에서 update
                    progress_bar.set_postfix({"Loss": running_avg_loss,
                                              "Accuracy": accuracy})

        if not isinstance(self.scheduler, torch.optim.lr_scheduler.ReduceLROnPlateau):
            self.scheduler.step()
            self.current_lr = self.scheduler.get_last_lr()[0]

        return running_avg_loss, accuracy

    def validate_epoch(self, epoch):
        if not self.val_loader:
            return None

        self.model.eval()

        # running 평균 loss 계산.
        accu_loss = 0
        running_avg_loss = 0
        # 정확도, 정확도 계산을 위한 전체 건수 및 누적 정확건수
        num_total = 0.0
        accu_num_correct = 0.0
        accuracy = 0.0
        
        with tqdm(total=len(self.val_loader), desc=f"Epoch {epoch+1} [Validating]", leave=True) as progress_bar:
            with torch.no_grad():
                for batch_idx, (inputs, targets) in enumerate(self.val_loader):
                    inputs = inputs.to(self.device)
                    targets = targets.to(self.device)

                    outputs = self.model(inputs)

                    loss = self.loss_fn(outputs, targets)
                    # batch 반복 시 마다 누적  loss를 구하고 이를 batch 횟수로 나눠서 running 평균 loss 구함.
                    accu_loss += loss.item()
                    running_avg_loss = accu_loss /(batch_idx + 1)

                    # accuracy metric 계산
                    # outputs 출력 예측 class값과 targets값 일치 건수 구하고
                    num_correct = (outputs.argmax(-1) == targets).sum().item()
                    # 배치별 누적 전체 건수와 누적 전체 num_correct 건수로 accuracy 계산
                    num_total += inputs.shape[0]
                    accu_num_correct += num_correct
                    accuracy = accu_num_correct / num_total

                    #tqdm progress_bar에 진행 상황 및 running 평균 loss와 정확도 표시
                    progress_bar.update(1)
                    if batch_idx % 20 == 0 or (batch_idx + 1) == progress_bar.total:  # 20 batch 횟수마다 또는 맨 마지막 batch에서 update
                        progress_bar.set_postfix({"Loss": running_avg_loss,
                                                  "Accuracy":accuracy})
        # scheduler에 검증 데이터 기반에서 epoch레벨로 계산된 loss를 입력해줌.
        if isinstance(self.scheduler, torch.optim.lr_scheduler.ReduceLROnPlateau):
            self.scheduler.step(running_avg_loss)
            self.current_lr = self.scheduler.get_last_lr()[0]

        return running_avg_loss, accuracy

    def fit(self, epochs):
        # epoch 시마다 학습/검증 결과를 기록하는 history dict 생성. learning rate 추가
        history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': [], 'lr': []}
        for epoch in range(epochs):
            train_loss, train_acc = self.train_epoch(epoch)
            val_loss, val_acc = self.validate_epoch(epoch)
            print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f} Train Accuracy: {train_acc:.4f}",
                  f", Val Loss: {val_loss:.4f} Val Accuracy: {val_acc:.4f}" if val_loss is not None else "",
                  f", Current lr:{self.current_lr:.6f}")
            # epoch 시마다 학습/검증 결과를 기록. learning rate 추가
            history['train_loss'].append(train_loss); history['train_acc'].append(train_acc)
            history['val_loss'].append(val_loss); history['val_acc'].append(val_acc)
            history['lr'].append(self.current_lr)

        return history

    # 학습이 완료된 모델을 return
    def get_trained_model(self):
        return self.model

In [ ]:
import torch
import torch.nn as nn
from torch.optim import SGD, Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau

NUM_INPUT_CHANNELS = 3
# 개(target값 0)와 고양이(target값 1) 2개
NUM_CLASSES = 2

model = create_resnet_model('resnet50', num_classes=NUM_CLASSES) #resnet50, resnet101
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
optimizer = Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()
scheduler = ReduceLROnPlateau(
            optimizer=optimizer, mode='min', factor=0.5, patience=3, threshold=0.01, min_lr=0.00001)

trainer = Trainer(model=model, loss_fn=loss_fn, optimizer=optimizer,
       train_loader=tr_loader, val_loader=val_loader, scheduler=scheduler, device=device)
# 학습 및 평가
history = trainer.fit(30)

### Predictor 클래스 생성 및 평가. 

In [ ]:
class Predictor:
    def __init__(self, model, device):
        self.model = model.to(device)
        self.device = device

    def evaluate(self, loader):
        self.model.eval()
        eval_metric = 0.0
        
        num_total = 0.0
        accu_num_correct = 0.0

        with tqdm(total=len(loader), desc=f"[Evaluating]", leave=True) as progress_bar:
            with torch.no_grad():
                for batch_idx, (inputs, targets) in enumerate(loader):
                    inputs = inputs.to(self.device)
                    targets = targets.to(self.device)
                    pred = self.model(inputs)

                    # 정확도 계산을 위해 누적 전체 건수와 누적 전체 num_correct 건수 계산  
                    num_correct = (pred.argmax(-1) == targets).sum().item()
                    num_total += inputs.shape[0]
                    accu_num_correct += num_correct
                    eval_metric = accu_num_correct / num_total

                    progress_bar.update(1)
                    if batch_idx % 20 == 0 or (batch_idx + 1) == progress_bar.total:
                        progress_bar.set_postfix({"Accuracy": eval_metric})
        
        return eval_metric

    def predict_proba(self, inputs):
        self.model.eval()
        with torch.no_grad():
            inputs = inputs.to(self.device)
            outputs = self.model(inputs)
            #예측값을 반환하므로 targets은 필요 없음.
            #targets = targets.to(self.device)
            pred_proba = F.softmax(outputs, dim=-1) #또는 dim=1

        return pred_proba

    def predict(self, inputs):
        pred_proba = self.predict_proba(inputs)
        pred_class = torch.argmax(pred_proba, dim=-1)

        return pred_class

In [ ]:
test_df = data_df[data_df['dataset']=='test']
test_image_paths = test_df['path'].to_list()
test_targets = test_df['target'].to_list()

IMG_SIZE=224
transform_02 = T.Compose([
                        T.Resize(size=(IMG_SIZE, IMG_SIZE)),
                        T.ToTensor(), 
                        T.Normalize(mean=[0.485, 0.456, 0.406], 
                                    std=[0.229, 0.224, 0.225])
])

test_dataset = CnD_Dataset(image_paths=test_image_paths, 
                            targets=test_targets, transform=transform_02)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)

trained_model = trainer.get_trained_model()

predictor = Predictor(model=trained_model, device=device)
eval_metric = predictor.evaluate(test_loader)
print(f'test dataset evaluation:{eval_metric:.4f}')

In [ ]:
import matplotlib.pyplot as plt

print(test_df['path'][0], test_df['target'][0], test_df['label'][0])
pil_image = Image.open(test_df['path'][0])

plt.imshow(pil_image)

In [ ]:
from PIL import Image

print(test_df['path'][0], test_df['target'][0], test_df['label'][0])
pil_image = Image.open(test_df['path'][0])

pil_tensor = transform_02(pil_image)
pil_tensor = pil_tensor.unsqueeze(0)

pred_class = predictor.predict(pil_tensor).item()
print('predicted class:', pred_class)

In [ ]:
# 10개의 테스트 이미지로 예측 결과
for i in range(10):
    pil_image = Image.open(test_df['path'][i])
    pil_tensor = transform_02(pil_image).unsqueeze(0)
    pred_class = predictor.predict(pil_tensor).item()
    print('actual class:', test_df['target'][i],'predicted class:', pred_class)